In [ ]:
import warnings
warnings.filterwarnings("ignore")  # Suppress all warnings

import os
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
import pickle
from tqdm import tqdm
import torch
import torch.nn as nn
from transformers import Wav2Vec2Model, Wav2Vec2Config


# ✅ Force TensorFlow to run on CPU (Prevents GPU memory usage)
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Load YAMNet model
yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

def extract_embedding(file_path):
    waveform, sr = librosa.load(file_path, sr=16000)
    waveform = waveform.astype(np.float32)
    scores, embeddings, spectrogram = yamnet_model(waveform)
    tf.keras.backend.clear_session()  # ✅ Free memory
    return np.mean(embeddings.numpy(), axis=0)

# Load trained model
model = tf.keras.models.load_model("yamnet_finetuned.h5")

# Test our trained model on random audio file
filename = "Datasets/Cry/0c8f14a9-6999-485b-97a2-913c1cbf099c-1430760379259-1.7-m-26-hu.wav"
embedding = extract_embedding(filename)
embedding = np.expand_dims(embedding, axis=0)
prediction = model.predict(embedding)
predicted_class = np.argmax(prediction)

# Load class mapping
CLASS_MAPPING = {0: "Cry", 1: "Screaming", 2: "NotScreaming"}

print(f"Predicted Class: {CLASS_MAPPING[predicted_class]}")
print(f"Prediction Scores: {prediction[0]}")

2025-02-24 04:42:52.630700: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 04:42:52.638428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740352372.647192    8676 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740352372.649855    8676 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 04:42:52.659022: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Class: Cry
Prediction Scores: [9.9248135e-01 7.2007808e-03 3.1792070e-04]
